In [9]:
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

In [2]:
paper_fetcher = PubmedQueryRun()

In [3]:
tools = [paper_fetcher]

In [4]:
load_dotenv()
llm = ChatOpenAI(temperature=0, model="gpt-4o", api_key=os.environ.get("KEY"))

In [5]:
tooled_model = llm.bind_tools(tools)

In [6]:
response = tooled_model.invoke([HumanMessage(content="Can you tell me the effects of opiates?")])

In [7]:
print(response)
print(response.content)

content='' additional_kwargs={'tool_calls': [{'id': 'call_uRGjlVQ59IApq0mtwzOEhya3', 'function': {'arguments': '{"query":"effects of opiates"}', 'name': 'pub_med'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 92, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_831e067d82', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-6bd22c65-3b1f-4271-840e-3ffdcec618ea-0' tool_calls=[{'name': 'pub_med', 'args': {'query': 'effects of opiates'}, 'id': 'call_uRGjlVQ59IApq0mtwzOEhya3', 'type': 'tool_call'}] usage_metadata={'input_tokens': 92, 'output_tokens': 18, 'total_tokens': 110, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0

In [10]:
agent_executor = create_react_agent(llm, tools)

In [12]:
agent_executor.invoke({"messages": [HumanMessage(content="What genes inherently cause overdose?")]})    

Too Many Requests, waiting for 0.40 seconds...


{'messages': [HumanMessage(content='What genes inherently cause overdose?', additional_kwargs={}, response_metadata={}, id='9ed73ae1-242f-40d1-98a6-3270294699f8'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UzfP7qc4m5Kl7vS1eWEcMa0Z', 'function': {'arguments': '{"query":"genes associated with overdose risk"}', 'name': 'pub_med'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 87, 'total_tokens': 105, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_831e067d82', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f7f82808-477c-4095-b4de-d00e3b492209-0', tool_calls=[{'name': 'pub_med', 'args': {'query': 'genes associated with overdose risk'}, 'id': 'call_UzfP7qc4m5Kl7vS1eW